In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/train.csv


In [44]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [208]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [209]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [210]:
df = test

In [211]:
df['location'] = df.location.apply(lambda x: '' if x != x else x.lower())
df.loc[df.location == 'new York, ny','location'] = 'new york'
df.loc[df.location == 'nyc','location'] = 'new york'
df.loc[df.location == 'ny','location'] = 'new york'
df.loc[df.location == 'new york city','location'] = 'new york'
df.loc[df.location == 'us','location'] = 'united states'
df.loc[df.location == 'usa','location'] = 'united states'
df.loc[df.location == 'california','location'] = 'united states'
df.loc[df.location == 'southern california','location'] = 'united states'
df.loc[df.location == 'orange county','location'] = 'united states'
df.loc[df.location == 'u.s.a','location'] = 'united states'
df.loc[df.location == 'ca','location'] = 'united states'
df.loc[df.location == 'san francisco bay area','location'] = 'san francisco'
df.loc[df.location == 'uk','location'] = 'united kingdom'

In [212]:
df.loc[df.location == 'ss','location'] = ''
df.loc[df.location == 'everywhere','location'] = ''
df.loc[df.location == 'earth','location'] = ''
df.loc[df.location == '304','location'] = ''
df.loc[df.location == 'world','location'] = ''
df.loc[df.location == 'worldwide','location'] = ''
df.loc[df.location == '??????','location'] = np.nan
df.loc[df.location == 'pedophile hunting ground','location'] = ''
df.loc[df.location == 'planet earth','location'] = ''
df.loc[df.location == 'global','location'] = ''
df.loc[df.location == 'in the word of god','location'] = ''
df.loc[df.location == '?','location'] = ''
df.loc[df.location == 'mad as hell','location'] = ''
df.loc[df.location == '??','location'] = ''
df.loc[df.location == '?????','location'] = ''
df.loc[df.location == 'happily married with 2 kids','location'] = ''
df.loc[df.location == 'south','location'] = ''
df.loc[df.location == 'world wide','location'] = ''

In [213]:
df = df.replace(np.nan, '', regex=True)
df['keyword'] = df.keyword.str.replace('%20', '_')
df['keyword'] = df.keyword.str.lower()
df['text'] = df['keyword'] + ' ' + df['location'] + ' ' + df['text']

In [214]:
df.sample(10)

,id,keyword,location,text
21,70,ablaze,washington state,ablaze washington state Burning Man Ablaze! by...
1271,4187,drown,east coast,drown east coast drown me in clementines
2489,8308,rubble,on,rubble on China's Stock Market Crash: Are Ther...
1758,5941,hazard,mystic falls,hazard mystic falls There will be a 3-match ba...
2396,8011,refugees,kansas ks,refugees kansas ks 'imagine an entire aisle d...
2887,9549,threat,"portland, oregon, usa","threat portland, oregon, usa Most Popular Cont..."
1533,5111,fatal,cincinnati,fatal cincinnati Protesters mark year since fa...
651,2130,catastrophe,"carrboro, nc","catastrophe carrboro, nc @ShaneRyanHere CATAST..."
187,602,arsonist,,arsonist mo the way she says 'carry' https://...
3254,10845,,,Officials: Alabama home quarantined over pos...


In [215]:
df = train
df = df.drop(['id','keyword','location'], axis=1)

In [117]:
df.sample(8)

,text,target
713,Threw a chicken nugget at my sisters lip and n...,0
5453,New technology designed to help prevent danger...,0
4108,Nice job Calgary transit ! http://t.co/RGOGUyt0LF,0
6350,@sabcnewsroom sabotage!I rule out structural f...,0
1140,#Australia #News ; #Japan marks 70th anniversa...,1
1958,GREAT CONDITION Easton Cyclone Softball Bat Fa...,1
5597,The Latest: More Homes Razed by Northern Calif...,1
7068,A look at state actions a year after Ferguson'...,0


In [216]:
def clean_str(string):
    string = re.sub(r'https?\://\S+', '', string)
    string = re.sub(r'http?\://\S+', '', string)
    string = re.sub(r'@\w*\s', '', string)
    string = re.sub(r'#\w*\s', '', string)
    string = re.sub(r'\d', '', string)
    return string

df['text'] = df['text'].apply(lambda x: clean_str(str(x)))

In [119]:
df.sample(8)

,text,target
5198,God the are so cocky right now and I love it. ...,0
3243,He came to a land which was engulfed in tribal...,1
7127,nside a Dragon's belly. Or an ice cave under\n...,1
3755,This fire is WAY too close wtf is going on ????,1
1311,Don't be so modest. You certainly... *sniff* *...,0
6966,Amen today is the Day of Salvation. THX brothe...,1
5848,like why on earth would you want anybody to be...,0
1862,When you're girlfriend is completely gorgeous?...,0


In [217]:
# Step - a : Remove blank rows if any.
df['text'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
df['text'] = [entry.lower() for entry in df['text']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
df['text']= [word_tokenize(entry) for entry in df['text']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(df['text']):
    # Declaring Empty List to train the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    df.loc[index,'text_final'] = str(Final_words)

In [123]:
df.sample(8)

,text,target,text_final
1076,"[hiroshima, marks, years, since, bomb]",1,"['hiroshima', 'mark', 'year', 'since', 'bomb']"
4636,"[st, wk, of, rainier, diet, and, my, street, s...",0,"['st', 'wk', 'rainy', 'diet', 'street', 'sewar..."
5876,"[watch, this, airport, get, swallowed, up, by,...",1,"['watch', 'airport', 'get', 'swallow', 'sandst..."
5879,"[watch, this, airport, get, swallowed, up, by,...",1,"['watch', 'airport', 'get', 'swallow', 'sandst..."
4297,"[then, i, do, this, to, one, of, them, ?, ?, ?...",0,['one']
7310,"[in, europe, nature, is, kind, while, in, us, ...",1,"['europe', 'nature', 'kind', 'u', 'tornados', ..."
7253,"['the, reagan, administration, had, arranged, ...",1,"['reagan', 'administration', 'arrange', 'israe..."
6306,"[how, to, freeze, fruits, and, veggies]",0,"['freeze', 'fruit', 'veggie']"


In [249]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df['text_final'],df['target'],test_size=0.33)

In [202]:
Tfidf_vect = TfidfVectorizer(max_features=7000)
Tfidf_vect.fit(df['text_final'])
X_train_Tfidf = Tfidf_vect.transform(X_train)

In [240]:
X_test_Tfidf = Tfidf_vect.transform(df['text_final'])

In [12]:
Tfidf_vect.vocabulary_

{'deed': 1113,
 'reason': 3564,
 'earthquake': 1351,
 'may': 2692,
 'allah': 110,
 'forgive': 1722,
 'forest': 1718,
 'fire': 1659,
 'near': 2938,
 'la': 2440,
 'canada': 657,
 'resident': 3646,
 'ask': 250,
 'place': 3291,
 'officer': 3058,
 'evacuation': 1486,
 'shelter': 3916,
 'order': 3113,
 'expect': 1528,
 'people': 3229,
 'receive': 3567,
 'wildfire': 4861,
 'california': 641,
 'get': 1836,
 'send': 3868,
 'photo': 3258,
 'alaska': 100,
 'smoke': 4017,
 'pour': 3361,
 'school': 3819,
 'rockyfire': 3717,
 'update': 4649,
 'hwy': 2126,
 'close': 828,
 'direction': 1214,
 'due': 1326,
 'lake': 2447,
 'county': 978,
 'flood': 1689,
 'disaster': 1223,
 'heavy': 2019,
 'rain': 3515,
 'cause': 700,
 'flash': 1679,
 'flooding': 1690,
 'street': 4186,
 'colorado': 872,
 'spring': 4121,
 'area': 220,
 'top': 4473,
 'hill': 2048,
 'see': 3853,
 'wood': 4900,
 'emergency': 1411,
 'happen': 1973,
 'building': 602,
 'across': 35,
 'afraid': 72,
 'tornado': 4477,
 'come': 879,
 'three': 4415,

In [14]:
print(X_train_Tfidf)

  (0, 4487)	0.3442749755035165
  (0, 4455)	0.46117750517795797
  (0, 3267)	0.25706795883129396
  (0, 2402)	0.20867123480156785
  (0, 2102)	0.07122970533028694
  (0, 2016)	0.26508493709468134
  (0, 1752)	0.26812702503944696
  (0, 1573)	0.2438698650169717
  (0, 1303)	0.29302966230605093
  (0, 858)	0.31760779072888007
  (0, 822)	0.2761917884938696
  (0, 388)	0.29905037551963903
  (1, 3689)	0.27466036459446197
  (1, 3504)	0.40603229836009896
  (1, 3406)	0.40603229836009896
  (1, 2102)	0.08061317771861104
  (1, 1584)	0.31867813374868426
  (1, 1177)	0.30588701332205687
  (1, 643)	0.5506473697226985
  (1, 126)	0.3000060023476485
  (2, 4870)	0.3322394232847937
  (2, 4368)	0.35883075574849194
  (2, 3855)	0.35409516558516363
  (2, 3752)	0.345748433319022
  (2, 2237)	0.3799374837404157
  :	:
  (5325, 560)	0.5183175421932857
  (5325, 534)	0.5047924414430615
  (5326, 4867)	0.3805269949464628
  (5326, 4844)	0.36487315136308085
  (5326, 3902)	0.5119176271239588
  (5326, 3738)	0.43666975338872016
  (5

In [205]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train_Tfidf,y_train)

MultinomialNB()

In [204]:
print("Naive Bayes Accuracy Score on test : ",accuracy_score(Naive.predict(X_test_Tfidf), y_test))

Naive Bayes Accuracy Score on test :  0.7978511738957421


In [206]:
SVM = svm.SVC(C=1.0, kernel='sigmoid', degree=10, gamma='scale')
SVM.fit(X_train_Tfidf,y_train)

SVC(degree=10, kernel='sigmoid')

In [207]:
print("SVM Accuracy Score on test : ",accuracy_score(SVM.predict(X_test_Tfidf), y_test))

SVM Accuracy Score on test :  0.8014325507361719


In [241]:
predictions_SVM = SVM.predict(X_test_Tfidf)

In [242]:
predictions_SVM = pd.DataFrame(predictions_SVM, columns=[ 'target'] )

In [236]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [243]:
predictions_SVM

,target
0,0
1,1
2,1
3,1
4,0
5,1
6,1
7,0
8,1
9,1


In [238]:
test

,id,keyword,location,text,target
0,0,NaN,,Just happened a terrible car crash,0.0
1,2,NaN,,"Heard about #earthquake is different cities, s...",0.0
2,3,NaN,,"there is a forest fire at spot pond, geese are...",0.0
3,9,NaN,,Apocalypse lighting. #Spokane #wildfires,0.0
4,11,NaN,,Typhoon Soudelor kills 28 in China and Taiwan,0.0
5,12,NaN,,We're shaking...It's an earthquake,1.0
6,21,NaN,,They'd probably still show more life than Arse...,0.0
7,22,NaN,,Hey! How are you?,1.0
8,27,NaN,,What a nice hat?,0.0
9,29,NaN,,Fuck off!,0.0


In [244]:
test['target'] = predictions_SVM['target']

In [245]:
results = test[['id', 'target']]

In [246]:
results.sample(10)

,id,target
3104,10277,0
690,2247,0
2195,7347,1
2877,9528,1
1937,6526,0
1448,4805,0
963,3206,0
2055,6901,0
2432,8127,1
1667,5623,0


In [248]:
results.to_csv('results.csv', index=False)